In [15]:
import tensorflow as tf 

# Introduction

In this tutorial, we'll be introducing neural networks and looking at how to classify digits with these machine learning models. As with all supervised learning methods, we have some (input, output) pairs which we denote as $ (x_i, y_i) $ and we have $n$ of these, so $i \in [1...n]$. We want to learn a function $f: x \rightarrow{} y$ that maps inputs to outputs. 

Today, we're going to look at classifying digits. The inputs will be a 28 x 28 black and white image, and the output will be a 10 dimensional vector that represents the probabilities for each of the 10 digits. From these probabilities, you can then choose the probability with the highest value to produce a single value that best representing the number in the image. 

![caption](Data/MNISTNN.png)

The dataset we're going to be using is called MNIST. MNIST is a dataset of over 60,000 handwritten digits and the corresponding labels of the digits the image represents. It is easily the most used dataset in machine learning. We'll start this tutorial by using the Tensorflow API to load in the dataset. 

In [16]:
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Network Creation

Let's first define some parameters.

In [17]:
numClasses = 10 # The number of categories the model is choosing from
inputSize = 784 # A 28x28 image will have 784 total pixel values
numHiddenUnits = 50 # Number of hidden units this one layer NN will have
trainingIterations = 10000 # Number of times the training loop is run
batchSize = 100 # Represents how many images we feed in one training batch

Now we'll start by defining placeholders for our inputs and outputs. The purpose of a placeholder is basically to tell Tensorflow "We're going to input in our input data and labels vector later, but for now, we're going to define these placeholder variables instead". It lets Tensorflow know about the size of the inputs and labels beforehand. The shape of X will be None x inputSize and None x numClasses for y (our labels). The None keyword means that the value can be determined at session runtime. We normally have None as our first dimension so that we can have variable batch sizes (With a batch size of 100, the input to the network would be 100 x 784). With the None keywoard, we don't have to specify batch_size until later. 

In [18]:
tf.reset_default_graph() # Just to make sure that we start with a new graph
X = tf.placeholder(tf.float32, shape = [None, inputSize])
y = tf.placeholder(tf.float32, shape = [None, numClasses])

Next, we will define the weight matrices and bias terms. The key when defining these variable is to keep track of the shapes and to make sure that the output matrix of the previous layer is able to multiply with the weight matrix of the next layer. 

In [19]:
W1 = tf.Variable(tf.truncated_normal([inputSize, numHiddenUnits], stddev=0.1))
B1 = tf.Variable(tf.constant(0.1), [numHiddenUnits])
W2 = tf.Variable(tf.truncated_normal([numHiddenUnits, numClasses], stddev=0.1))
B2 = tf.Variable(tf.constant(0.1), [numClasses])

Now, we will define the matrix multiplication operations and follow each matrix multiply with a non linear layer

In [20]:
hiddenLayerOutput = tf.matmul(X, W1) + B1
hiddenLayerOutput = tf.nn.relu(hiddenLayerOutput)
finalOutput = tf.matmul(hiddenLayerOutput, W2) + B2
finalOutput = tf.nn.relu(finalOutput)

![caption](Data/NNStructure.png)

As you can see with the above image, the final 10 dimensional output will be fed through a softmax layer to obtain probabilities. Instead of defining the softmax function explicity, we'll be using Tensorflow's function tf.nn.softmax_cross_entropy_with_logits function to perform the softmax + the cross entropy loss all in one line. 

# Loss Function

Now that we know how the input gets fed through the network, we can analyze the output. The output is a vector that represents the probabilities for the classes. We then want to compare this prediction with the label, and then minimize the loss using a Tensorflow optimizer. 

In [21]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = finalOutput))
opt = tf.train.GradientDescentOptimizer(learning_rate = .1).minimize(loss)

The following are the variables that help with calculating accuracy. The correct prediction variable will take a look at the output vector, find the probability with the highest value, and return its index. This is then compared with the actual lables. 

In [22]:
correct_prediction = tf.equal(tf.argmax(finalOutput,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

# Training

Here's the part where we start training our model. We'll load in our training set by calling Tensorflow's mnist.train.next_batch() function. The sess.run function has two arguments. The first is called the "fetches" argument. It defines the value for you're interested in computing/running. For example, in our case, we want to both run the optimizer object so that the cross entropy loss gets minimized and evaluate the current loss value (in case we want to print that value). Therefore, we'll use [opt, loss] for our first argument. The second argument is where we input our feed_dict. This data structure is where we provide values to all of our placeholders. We repeat this for a set number of iterations.

In [23]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

for i in range(trainingIterations):
    batch = mnist.train.next_batch(batchSize)
    batchInput = batch[0]
    batchLabels = batch[1]
    _, trainingLoss = sess.run([opt, loss], feed_dict={X: batchInput, y: batchLabels})
    if i%1000 == 0:
        trainAccuracy = accuracy.eval(session=sess, feed_dict={X: batchInput, y: batchLabels})
        print ("step %d, training accuracy %g"%(i, trainAccuracy))

step 0, training accuracy 0.12
step 1000, training accuracy 0.83
step 2000, training accuracy 0.96
step 3000, training accuracy 0.97
step 4000, training accuracy 0.98
step 5000, training accuracy 0.99
step 6000, training accuracy 0.97
step 7000, training accuracy 0.97
step 8000, training accuracy 0.99
step 9000, training accuracy 0.97


# Testing

Now, we will test the network on data that it has never seen before. 

In [25]:
testInputs = mnist.test.images
testLabels = mnist.test.labels
acc = accuracy.eval(session=sess, feed_dict = {X: testInputs, y: testLabels})
print("testing accuracy: {}".format(acc))

testing accuracy: 0.9675999879837036


# 2 Layer Neural Net

Now, let's look at the accuracy we get when we define a two layer neural net. 

In [28]:
numHiddenUnitsLayer2 = 100
trainingIterations = 10000
tf.reset_default_graph() 

X = tf.placeholder(tf.float32, shape = [None, inputSize])
y = tf.placeholder(tf.float32, shape = [None, numClasses])

W1 = tf.Variable(tf.random_normal([inputSize, numHiddenUnits], stddev=0.1))
B1 = tf.Variable(tf.constant(0.1), [numHiddenUnits])
W2 = tf.Variable(tf.random_normal([numHiddenUnits, numHiddenUnitsLayer2], stddev=0.1))
B2 = tf.Variable(tf.constant(0.1), [numHiddenUnitsLayer2])
W3 = tf.Variable(tf.random_normal([numHiddenUnitsLayer2, numClasses], stddev=0.1))
B3 = tf.Variable(tf.constant(0.1), [numClasses])

hiddenLayerOutput = tf.matmul(X, W1) + B1
hiddenLayerOutput = tf.nn.relu(hiddenLayerOutput)
hiddenLayer2Output = tf.matmul(hiddenLayerOutput, W2) + B2
hiddenLayer2Output = tf.nn.relu(hiddenLayer2Output)
finalOutput = tf.matmul(hiddenLayer2Output, W3) + B3

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = finalOutput))
opt = tf.train.GradientDescentOptimizer(learning_rate = .1).minimize(loss)

correct_prediction = tf.equal(tf.argmax(finalOutput,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

for i in range(trainingIterations):
    batch = mnist.train.next_batch(batchSize)
    batchInput = batch[0]
    batchLabels = batch[1]
    _, trainingLoss = sess.run([opt, loss], feed_dict={X: batchInput, y: batchLabels})
    if i%1000 == 0:
        train_accuracy = accuracy.eval(session=sess, feed_dict={X: batchInput, y: batchLabels})
        print ("step %d, training accuracy %g"%(i, train_accuracy))

testInputs = mnist.test.images
testLabels = mnist.test.labels
acc = accuracy.eval(session=sess, feed_dict = {X: testInputs, y: testLabels})
print("testing accuracy: {}".format(acc))

step 0, training accuracy 0.17
step 1000, training accuracy 0.97
step 2000, training accuracy 0.98
step 3000, training accuracy 1
step 4000, training accuracy 0.99
step 5000, training accuracy 1
step 6000, training accuracy 0.99
step 7000, training accuracy 1
step 8000, training accuracy 1
step 9000, training accuracy 1
testing accuracy: 0.9740999937057495


# Additional Resources

* Stanford's Neural Network [Tutorial](http://ufldl.stanford.edu/tutorial/supervised/MultiLayerNeuralNetworks/)
* Michael Neilson's [Online Book](http://neuralnetworksanddeeplearning.com/)
* Andrej Karpathy's [Blog Post](http://karpathy.github.io/neuralnets/ )